In [21]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from skimage import morphology
import os
import random

In [22]:
root = "Z:/Projectline_modelling_lung_cancer/Nii_Processed2/"

In [23]:
PxList = os.listdir(root)
len(PxList)

11

In [24]:
def LookFiles_Snapshot(folder_path):
    planCT_list,planCTLung_list = ([],[])
    bp50_list, bp50Lung_list = ([],[])
    gtv_list, itv_list = ([],[])
    for root,dirs,files in os.walk(folder_path):
        for f in files:
            if "PlanCT_CTProcessed.nii" in f:
                planCT_list.append(os.path.join(folder_path,f))
            if "PlanCT_LungProcessed.nii" in f:
                planCTLung_list.append(os.path.join(folder_path,f))
            if "bp50_CTProcessed.nii" in f:
                bp50_list.append(os.path.join(folder_path,f))
            if "bp50_LungProcessed.nii" in f:
                bp50Lung_list.append(os.path.join(folder_path,f))
            if "GTV" in f:
                gtv_list.append(os.path.join(folder_path,f))
            if "ITV" in f:
                itv_list.append(os.path.join(folder_path,f))
    
    #print("Len of Files",len(planCT_list),len(planCTLung_list),len(bp50_list),len(bp50Lung_list),len(gtv_list),len(itv_list))


    return planCT_list,planCTLung_list,bp50_list,bp50Lung_list,gtv_list,itv_list
    

In [25]:
def largestRatio(struct_np):
    ratioVal_x,ratioVal_y,ratioVal_z = 0,0,0
    listRatios_x,listRatios_y,listRatios_z = [],[],[]
    #Z
    maxRatio = 0
    for i in range(struct_np.shape[-1]):
        if np.sum(struct_np[:,:,i])>0:
            currPosSum = np.sum(struct_np[:,:,i])
            currTotSum = struct_np.shape[0]*struct_np.shape[1]
            currRatio = currPosSum/currTotSum
            listRatios_z.append(currRatio)
            if currRatio>maxRatio:
                maxRatio = currRatio
                maxSlide = i
    ratioVal_z = maxRatio
    maxSlide_z = maxSlide
    #Y
    maxRatio = 0
    for i in range(struct_np.shape[-2]):
        if np.sum(struct_np[:,i,:])>0:
            currPosSum = np.sum(struct_np[:,i,:])
            currTotSum = struct_np.shape[0]*struct_np.shape[2]
            currRatio = currPosSum/currTotSum
            listRatios_y.append(currRatio)
            if currRatio>maxRatio:
                maxRatio = currRatio
                maxSlide = i
    ratioVal_y = maxRatio
    maxSlide_y = maxSlide
    #X
    maxRatio = 0
    for i in range(struct_np.shape[-3]):
        if np.sum(struct_np[i,:,:])>0:
            currPosSum = np.sum(struct_np[i,:,:])
            currTotSum = struct_np.shape[1]*struct_np.shape[2]
            currRatio = currPosSum/currTotSum
            listRatios_x.append(currRatio)
            if currRatio>maxRatio:
                maxRatio = currRatio
                maxSlide = i
    ratioVal_x = maxRatio
    maxSlide_x = maxSlide

    return listRatios_x,listRatios_y,listRatios_z,ratioVal_x,ratioVal_y,ratioVal_z,maxSlide_x,maxSlide_y,maxSlide_z

In [26]:
listOfShapes_X,listOfShapes_Y,listOfShapes_Z = [],[],[]
gtvSums_list = []
listMaxRatios = []
listAvgRatios = []
listMedianRatios = []
okPx,errorPx = [],[]

for Px in PxList:
    planCT_list,planCTLung_list,bp50_list,bp50Lung_list,gtv_list,itv_list = LookFiles_Snapshot(root+Px)
    flagOkPx = False
    #ReadImages
    if False:
        ct_np  = nib.load(planCT_list[0]).get_fdata()
        lung_np  = nib.load(planCTLung_list[0]).get_fdata()
        bp50_np  = nib.load(bp50_list[0]).get_fdata()
        bp50Lung_np  = nib.load(bp50Lung_list[0]).get_fdata()
        aveLung_filled = FixHoles(lung_np)
        bp50Lung_filled = FixHoles(bp50Lung_np)

    if len(planCT_list)==0 and len(bp50_list)==0:
            errorPx.append(Px)
    elif len(itv_list)==0 and len(gtv_list)==0:
            errorPx.append(Px)
    elif len(gtv_list)>0:
        tum_np  = nib.load(gtv_list[0]).get_fdata()
        flagOkPx = True
    else:
        tum_np  = nib.load(itv_list[0]).get_fdata()
        flagOkPx = True
        

    if flagOkPx:
        #Calculations
        tum_filled = FixHoles(tum_np)
        if np.sum(tum_filled)!=0:
            okPx.append(Px)
            listOfShapes_X.append(tum_filled.shape[0])
            listOfShapes_Y.append(tum_filled.shape[1])
            listOfShapes_Z.append(tum_filled.shape[2])
            gtvSums_list.append(np.sum(tum_filled))

            listRatios_x,listRatios_y,listRatios_z,ratioVal_x,ratioVal_y,ratioVal_z,maxSlide_x,maxSlide_y,maxSlide_z = largestRatio(tum_filled)
        
            listMaxRatios.append([ratioVal_x,ratioVal_y,ratioVal_z])
            listAvgRatios.append([np.mean(listRatios_x),np.mean(listRatios_y),np.mean(listRatios_z)])
            listMedianRatios.append([np.median(listRatios_x),np.median(listRatios_y),np.median(listRatios_z)])
        else:
            errorPx.append(Px)

        
listOfUniquesShapes = [np.unique(listOfShapes_X),np.unique(listOfShapes_Y),np.unique(listOfShapes_Z)]


gtvSum_Avg = np.mean(gtvSums_list)
gtvSum_Median = np.median(gtvSums_list)
maxRatio_Avg = np.mean(listMaxRatios,0)
avgRatio_Avg = np.mean(listAvgRatios,0)
avgRatio_Med = np.median(listMedianRatios,0)

print("Unique Shapes",listOfUniquesShapes)
print("Avg GTV Mm3",gtvSum_Avg,"in ml ",gtvSum_Avg/1000)
print("Median GTV Mm3",gtvSum_Median,"in ml ",gtvSum_Median/1000)
print("Max Ratio Avg",maxRatio_Avg*100)
print("Avg Ratio Avg",avgRatio_Avg*100)
print("Avg Ratio Median", avgRatio_Med*100)
print("Num of Px",len(PxList),"Ok",len(okPx),"err",len(errorPx))


Unique Shapes [array([320]), array([320]), array([320])]
Avg GTV Mm3 56947.5 in ml  56.9475
Median GTV Mm3 30319.5 in ml  30.3195
Max Ratio Avg [1.50419922 1.28662109 1.37666016]
Avg Ratio Avg [0.78645735 0.70161881 0.78864796]
Avg Ratio Median [0.52197266 0.52197266 0.4934082 ]
Num of Px 11 Ok 10 err 1


In [27]:
print("Max Ratio Avg",maxRatio_Avg*100)
print("Avg Ratio Avg",avgRatio_Avg*100)
print("Avg Ratio Median", avgRatio_Med*100)

Max Ratio Avg [1.50419922 1.28662109 1.37666016]
Avg Ratio Avg [0.78645735 0.70161881 0.78864796]
Avg Ratio Median [0.52197266 0.52197266 0.4934082 ]


In [31]:
def Screenshot(ct_np,gtv_np,middle_x,middle_y,middle_z,contourBool=True,Px2Save=None,ImageType=None):
    minHU = -1024
    maxHU = 600
    savebool = True
    root2save= "Z:/Projectline_modelling_lung_cancer/JPEGs/Processed/V2/"
    if Px2Save is not None:
        pathToSave = os.path.join(root2save,Px2Save)
        if not(os.path.exists(pathToSave)):
            os.mkdir(pathToSave)
    
    print(middle_x,middle_y,middle_z,Px2Save,ct_np.shape,ImageType,gtv_np.shape)

    
    clipped_image = np.clip(ct_np, minHU, maxHU)
    #middle_x,middle_y,middle_z = getMiddleSlices(gtv_np)
    plt.figure(figsize=(15, 15), dpi=300)
    plt.imshow(clipped_image[:,:,middle_z],cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("Z:"+str(middle_z))
    if contourBool:
        plt.contour(gtv_np[:,:,middle_z],linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Axial.jpeg")
        plt.close()
    plt.figure(figsize=(15, 15), dpi=300)
    plt.imshow(np.rot90(clipped_image[:,middle_y,:]),cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("Y:"+str(middle_y))
    if contourBool:
        plt.contour(np.rot90(gtv_np[:,middle_y,:]),linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Sagital.jpeg")
        plt.close()
    plt.figure(figsize=(15, 15), dpi=300)
    plt.imshow(np.rot90(clipped_image[middle_x,:,:]),cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("X:"+str(middle_x))
    if contourBool:
        plt.contour(np.rot90(gtv_np[middle_x,:,:]),linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Coronal.jpeg")
        plt.close()

In [32]:

def CountSlidesPerCategory(struct_np,tumorSlide_Tresh):
    #Z
    countTum_z = 0
    countSemi_z =0
    indices_Tum_z = []
    indices_Semi_z = []
    for i in range(struct_np.shape[-1]):
        if np.sum(struct_np[:,:,i])>0:
            currRatio = np.sum(struct_np[:,:,i])/(struct_np.shape[0]*struct_np.shape[1])
            if currRatio > tumorSlide_Tresh[-1]:
                countTum_z+=1
                indices_Tum_z.append(i)
            elif currRatio > 0:
                countSemi_z+=1
                indices_Semi_z.append(i)
    #Y
    countTum_y = 0
    countSemi_y =0
    indices_Tum_y= []
    indices_Semi_y = []
    for i in range(struct_np.shape[-2]):
        if np.sum(struct_np[:,i,:])>0:
            currRatio = np.sum(struct_np[:,i,:])/(struct_np.shape[0]*struct_np.shape[2])
            if currRatio > tumorSlide_Tresh[-2]:
                countTum_y+=1
                indices_Tum_y.append(i)
            elif currRatio > 0:
                countSemi_y+=1
                indices_Semi_y.append(i)
    #X
    countTum_x = 0
    countSemi_x =0
    indices_Tum_x= []
    indices_Semi_x = []
    for i in range(struct_np.shape[-3]):
        if np.sum(struct_np[i,:,:])>0:
            currRatio = np.sum(struct_np[i,:,:])/(struct_np.shape[1]*struct_np.shape[2])
            if currRatio > tumorSlide_Tresh[-3]:
                countTum_x+=1
                indices_Tum_x.append(i)
            elif currRatio > 0:
                countSemi_x+=1
                indices_Semi_x.append(i)

    
    #return countTum_x,countTum_y,countTum_z,countSemi_x,countSemi_y,countSemi_z,
    return indices_Tum_x,indices_Tum_y,indices_Tum_z,indices_Semi_x,indices_Semi_y,indices_Semi_z

In [33]:
factor=.2
tumorSlide_AvgTresh = [avgRatio_Avg[0]*factor,avgRatio_Avg[1]*factor,avgRatio_Avg[2]*factor]
tumorSlide_MediTresh = [avgRatio_Med[0]*factor,avgRatio_Med[1]*factor,avgRatio_Med[2]*factor]

In [37]:
totTum_x,totTum_y,totTum_z = 0,0,0
totSemi_x,totSemi_y,totSemi_z = 0,0,0
count=0
for Px in okPx:
    Px = str(1007981)
    planCT_list,planCTLung_list,bp50_list,bp50Lung_list,gtv_list,itv_list = LookFiles_Snapshot(root+Px)
    
    if len(gtv_list)>0:
        tum_np  = nib.load(gtv_list[0]).get_fdata()
        ct_np  = nib.load(bp50_list[0]).get_fdata()
        tum_filled = FixHoles(tum_np)
        ContourType="GTV"
    else:
        tum_np  = nib.load(itv_list[0]).get_fdata()
        ct_np  = nib.load(planCT_list[0]).get_fdata()
        tum_filled = FixHoles(tum_np)
        ContourType="ITV"

    #countTum_x,countTum_y,countTum_z,countSemi_x,countSemi_y,countSemi_z = CountSlidesPerCategory(tum_filled,tumorSlide_MediTresh)
    indices_Tum_x,indices_Tum_y,indices_Tum_z,indices_Semi_x,indices_Semi_y,indices_Semi_z= CountSlidesPerCategory(tum_filled,tumorSlide_MediTresh)
    
    totTum_x+=len(indices_Tum_x)
    totTum_y+=len(indices_Tum_y)
    totTum_z+=len(indices_Tum_z)

    totSemi_x+=len(indices_Semi_x)
    totSemi_y+=len(indices_Semi_y)
    totSemi_z+=len(indices_Semi_z)

    if len(indices_Tum_x)!=0: randomcTumorX = random.choice(indices_Tum_x)
    else: randomcTumorX = 0
    if len(indices_Tum_y)!=0: randomcTumorY = random.choice(indices_Tum_y)
    else: randomcTumorY = 0
    if len(indices_Tum_z)!=0: randomcTumorZ = random.choice(indices_Tum_z)
    else: randomcTumorZ = 0 

    #Screenshot(ct_np,tum_filled,randomcTumorX,randomcTumorY,randomcTumorZ,False,Px,ContourType+"TumorOff")
    Screenshot(ct_np,tum_filled,randomcTumorX,randomcTumorY,randomcTumorZ,True,Px,ContourType+"Tumor")
    
    if len(indices_Semi_x)!=0: randomcTumorX = random.choice(indices_Semi_x)
    else: randomcTumorX = 0
    if len(indices_Semi_y)!=0: randomcTumorY = random.choice(indices_Semi_y)
    else: randomcTumorY = 0
    if len(indices_Semi_z)!=0: randomcTumorZ = random.choice(indices_Semi_z)
    else: randomcTumorZ = 0 
    
    #Screenshot(ct_np,tum_filled,randomcTumorX,randomcTumorY,randomcTumorZ,False,Px,ContourType+"SemiOff")
    Screenshot(ct_np,tum_filled,randomcTumorX,randomcTumorY,randomcTumorZ,True,Px,ContourType+"SemiTumor")

    break
    
print("Tot Tumor",totTum_x,totTum_y,totTum_z)
print("Tot SemiT",totSemi_x,totSemi_y,totSemi_z)

271 211 115 1007981 (500, 500, 310) ITVTumor (500, 500, 310)
242 194 109 1007981 (500, 500, 310) ITVSemiTumor (500, 500, 310)
Tot Tumor 43 33 39
Tot SemiT 7 5 5


In [20]:
ct_np.shape,tum_filled.shape

((320, 320, 320), (320, 320, 320))

In [42]:
#1007981 - Is incorrect

-1

In [43]:
indices_Tum_z

-1